# Data analysis

In [1]:
import os
import requests
import shutil
import tarfile

DATA_DIR_NAME = './dog_breeds_data'

# https://stackoverflow.com/questions/16694907/download-large-file-in-python-with-requests
def download_file(url, directory, filename):
    with requests.get(url, stream=True) as r:
        with open(os.path.join(directory, filename), 'wb') as f:
            shutil.copyfileobj(r.raw, f)
    return local_filename

def download_data():
    if not os.path.exists(DATA_DIR_NAME):
        os.makedirs('./dog_breeds_data')
    if not os.path.exists(os.path.join(DATA_DIR_NAME, 'images.tar')):
        download_file('http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar' , DATA_DIR_NAME, 'images.tar')
        extract_file(DATA_DIR_NAME, 'images.tar')
        os.remove(os.path.join(DATA_DIR_NAME, 'images.tar'))
    if not os.path.exists(os.path.join(DATA_DIR_NAME, 'annotation.tar')):
        download_file('http://vision.stanford.edu/aditya86/ImageNetDogs/annotation.tar' ,DATA_DIR_NAME, 'annotation.tar')
        extract_file(DATA_DIR_NAME, 'annotation.tar')
        os.remove(os.path.join(DATA_DIR_NAME, 'annotation.tar'))
    if not os.path.exists(os.path.join(DATA_DIR_NAME, 'lists.tar')):
        download_file('http://vision.stanford.edu/aditya86/ImageNetDogs/lists.tar' ,DATA_DIR_NAME, 'lists.tar')
        extract_file(DATA_DIR_NAME, 'lists.tar', extract_path=DATA_DIR_NAME+'/lists')
        os.remove(os.path.join(DATA_DIR_NAME, 'lists.tar'))

def extract_file(directory, filename, extract_path=DATA_DIR_NAME):
    shutil.unpack_archive(os.path.join(directory, filename), extract_path)


# download_data()


In [12]:
import scipy.io as sio

train = sio.loadmat(DATA_DIR_NAME+'/train_list.mat')
test = sio.loadmat(DATA_DIR_NAME+'/test_list.mat')
file = sio.loadmat(DATA_DIR_NAME+'/file_list.mat')

8580

In [13]:
from xml.dom import minidom